In [9]:
import os
import torch
import sys

import random
import torch
import cv2

import numpy as np
from pathlib import Path

# from YOLO.ultralytics_yolov5_master.models.experimental import attempt_load


# from models.experimental import attempt_load





# 设置缓存目录
cache_dir = 'YOLO'
torch.hub.set_dir(cache_dir)

colors = np.random.randint(125, 255, (80, 3))

# 定义模型权重文件路径
model_weights_path = 'YOLO/ultralytics_yolov5_master/yolov5s.pt'


# 加载模型
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model = attempt_load(model_weights_path)


# Model
# ../: 源目录, 查找 hubconf.py, 模型文件等，'yolov5s'：找模型文件，没有会下载  ; source='local' 本地加载模型
model = torch.hub.load('YOLO/ultralytics_yolov5_master', 'yolov5s', source='local')  # or yolov5m, yolov5l, yolov5x, custom


model.eval()  # 设置模型为评估模式


model = model.to(device)
print("model loaded")



# 加载图像（或视频帧）进行目标检测
image_path = 'bus.jpg'
img = cv2.imread(image_path)

# 进行目标检测
results = model(img)

# 显示检测结果
# results.show()
color = colors[int(random.randint(1, 10))]
print("color", color)

# 或者获取检测到的对象信息并进行后续处理
detected_objects = results.pandas().xyxy[0]
print(detected_objects)

for index, obj in detected_objects.iterrows():
    x1, y1, x2, y2, conf, label = int(obj[0]), int(obj[1]), int(obj[2]), int(obj[3]), obj[4], int(obj[5])
    print(f"x1:{x1} x2:{x2} y1:{y1} y2:{y2} conf:{conf} Label: {label}")
#     画出边界框
    cv2.rectangle(img, (x1, y1), (x2, y2), (int(color[0]), int(color[1]), int(color[2])), 2)

#     # 标签文本
    label_text = f"{model.names[label]}: {conf:.2f}"

#     # 在边界框上方显示类别标签和置信度
    cv2.putText(img, label_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (int(color[0]), int(color[1]), int(color[2])), 2)
    
cv2.imwrite('output.jpg', img)


YOLOv5 🚀 2023-11-26 Python-3.8.12 torch-1.11.0+cu113 CUDA:0 (NVIDIA GeForce RTX 3090, 24268MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


model loaded
color [238 237 189]
         xmin        ymin        xmax        ymax  confidence  class    name
0  219.181656  407.962585  346.653259  873.797974    0.864835      0  person
1   50.101345  395.412628  239.307892  913.109924    0.860520      0  person
2  673.481384  400.613281  810.000000  875.121704    0.849506      0  person
3   14.571888  219.987701  810.000000  809.948425    0.816634      5     bus
4    0.082597  553.358582   64.700478  874.117920    0.644922      0  person
x1:219 x2:346 y1:407 y2:873 conf:0.8648354411125183 Label: 0
x1:50 x2:239 y1:395 y2:913 conf:0.8605201244354248 Label: 0
x1:673 x2:810 y1:400 y2:875 conf:0.8495055437088013 Label: 0
x1:14 x2:810 y1:219 y2:809 conf:0.8166336417198181 Label: 5
x1:0 x2:64 y1:553 y2:874 conf:0.6449224948883057 Label: 0


True

In [23]:
import os
import torch
import sys
import random
import shutil

import cv2
import torch
import pdfplumber

import numpy as np
from pathlib import Path

# from YOLO.ultralytics_yolov5_master.models.experimental import attempt_load
# from models.experimental import attempt_load



def pre_process(images_path, pre_process_path):
    res_dict = dict()
    file_list = os.listdir(images_path)

    
    for idx, filename in enumerate(file_list):
        file_extension = filename.split(".")[-1]
        file_name = filename.split(".")[0]
        res_dict[filename] = {"file_type": file_extension.upper()}

        detection_type_list = ["pdf", "png", "jpg", "jpeg"]
        img_extension_list = ["png", "jpg", "jpeg"]
        if file_extension not in detection_type_list:
            continue
            
        if file_extension in img_extension_list:
            shutil.copy(os.path.join(images_path, filename), os.path.join(pre_process_path, f"{file_name}_page_0.png"))
        else:
            
            with pdfplumber.open(os.path.join(images_path, filename)) as pdf:
                for page_num, page in enumerate(pdf.pages):
                    img = page.to_image()
                    pdf_image = page.to_image().original
    
                    # 将原始图像数据转换为 OpenCV 图像对象
                    open_cv_image = cv2.cvtColor(np.array(pdf_image), cv2.COLOR_RGB2BGR)
                    cv2.imwrite(os.path.join(pre_process_path, f"{file_name}_page_{page_num}.png"), open_cv_image)
                
    return res_dict


def predict_image_class(task_id, images_path, pre_process_path, model, res_dict):
    # 加载图像并进行预处理
    pre_process_files = os.listdir(pre_process_path)
    pre_process_files_count = len(pre_process_files)
    result_path = os.path.join("output", "detect", task_id)
        
        
    colors = np.random.randint(125, 255, (80, 3))
    
    for idx, img_name in enumerate(pre_process_files):
        print(f"Task_id:{task_id} {idx}/{pre_process_files_count} img_name: {img_name}")

        file_extension = img_name.split(".")[-1]
        
        if file_extension != "png":
            continue
            
        file_name = img_name.split("_page")[0]
        
        matching_keys = [key for key in res_dict.keys() if file_name in key]
        if not matching_keys:
            continue
            
        related_pdf_name = matching_keys[0]

        img = cv2.imread(os.path.join(pre_process_path, img_name))
        
        # 进行目标检测
        results = model(img)
        
        # 显示检测结果
        # results.show()
        color = colors[int(random.randint(1, 10))]
#         print("color", color)
        
        # 或者获取检测到的对象信息并进行后续处理
        detected_objects = results.pandas().xyxy[0]
#         print(detected_objects)
        
        for index, obj in detected_objects.iterrows():
            x1, y1, x2, y2, conf, label = int(obj[0]), int(obj[1]), int(obj[2]), int(obj[3]), obj[4], int(obj[5])
#             print(f"x1:{x1} x2:{x2} y1:{y1} y2:{y2} conf:{conf} Label: {label}")
        #     画出边界框
    
            if conf < 0.5:
                continue
                
            cv2.rectangle(img, (x1, y1), (x2, y2), (int(color[0]), int(color[1]), int(color[2])), 2)
        
        #     # 标签文本
            label_text = f"{model.names[label]}: {conf:.2f}"
        
        #     # 在边界框上方显示类别标签和置信度
            cv2.putText(img, label_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (int(color[0]), int(color[1]), int(color[2])), 2)
            
            
        
            detection_res = {
                "class_id": int(label),
                "label": label_text,
                "bbox": [x1, y1, x2, y2],
                "score": conf,
            }

            if not res_dict[related_pdf_name].get("result"):
                res_dict[related_pdf_name]["result"] = list()
    
            res_dict[related_pdf_name]["result"].append(detection_res)
            
#             cv2.imwrite(os.path.join(result_path, 'output.jpg'), img)
        
            print(f'Task: {task_id} The file {img_name} predicted current object class is: {label_text} axis: {x1} {y1} {x2} {y2}')
            cv2.imwrite(os.path.join(result_path, f'{file_name}.jpg'), img)

        print(f'Task: {task_id} The file {img_name} predicted done.')

        
    return res_dict

        
def handler(detect_floder, task_id, node):
    # 设置缓存目录
    cache_dir = 'YOLO'
    torch.hub.set_dir(cache_dir)

    

    # 定义模型权重文件路径
    model_weights_path = 'YOLO/ultralytics_yolov5_master/yolov5s.pt'
    
    
    # 加载模型
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # model = attempt_load(model_weights_path)
    
    
    # Model
    # ../: 源目录, 查找 hubconf.py, 模型文件等，'yolov5s'：找模型文件，没有会下载  ; source='local' 本地加载模型
    model = torch.hub.load('YOLO/ultralytics_yolov5_master', 'yolov5s', source='local')  # or yolov5m, yolov5l, yolov5x, custom


    model.eval()  # 设置模型为评估模式


    model = model.to(device)
    print("model loaded")
    
    images_path = os.path.join("temp_storage", detect_floder)

    pre_process_path = os.path.join(images_path, "pre_process")
    if not os.path.exists(pre_process_path):
        os.mkdir(pre_process_path)

    # 预测图像类别

    res_dict = pre_process(images_path, pre_process_path)
    res = predict_image_class(task_id, images_path, pre_process_path, model, res_dict)
    print(res)
    
    
    # Todo: 释放资源， 结果增加到 MongoDB

    
if __name__ == "__main__":
    detect_floder = "detect_demo1"
    task_id = "123"
    node = "worker1"
    handler(detect_floder, task_id, node)








YOLOv5 🚀 2023-11-26 Python-3.8.12 torch-1.11.0+cu113 CUDA:0 (NVIDIA GeForce RTX 3090, 24268MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


model loaded
Task_id:123 0/13 img_name: table1_page_2.png
Task: 123 The file table1_page_2.png predicted done.
Task_id:123 1/13 img_name: test_page_0.png
Task: 123 The file test_page_0.png predicted current object class is: tie: 0.60 axis: 394 232 432 262
Task: 123 The file test_page_0.png predicted current object class is: person: 0.58 axis: 343 31 520 278
Task: 123 The file test_page_0.png predicted done.
Task_id:123 2/13 img_name: table1_page_1.png
Task: 123 The file table1_page_1.png predicted done.
Task_id:123 3/13 img_name: table2_page_0.png
Task: 123 The file table2_page_0.png predicted done.
Task_id:123 4/13 img_name: 不动产登记申请表_page_1.png
Task: 123 The file 不动产登记申请表_page_1.png predicted done.
Task_id:123 5/13 img_name: table2_page_1.png
Task: 123 The file table2_page_1.png predicted done.
Task_id:123 6/13 img_name: 02-身份证_page_0.png
Task: 123 The file 02-身份证_page_0.png predicted current object class is: clock: 0.89 axis: 478 179 542 242
Task: 123 The file 02-身份证_page_0.png predi